Project code



In [89]:
import requests
from bs4 import BeautifulSoup
import pandas as pd



url = 'https://www.uksport.gov.uk/our-work/investing-in-sport/historical-funding-figures'
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')

# Find the first line to then copy after
heading = soup.find('h3', string="Summer Olympic Sports")

# Get the table after the heading
table = heading.find_next('table') 

rows = table.find_all('tr')

# Append and then into pd dataframe

funding = []
for row in rows:
    cells = row.find_all(['th', 'td'])
    values = [cell.get_text(strip=True) for cell in cells]
    if values: 
        funding.append(values)

funding_df = pd.DataFrame(funding[1:], columns=funding[0])
funding_df


,Sport,London,Rio,Tokyo**,Paris*
0,Archery,"£3,479,900","£2,473,437","£1,122,879","£2,632,674"
1,Artistic Swimming,"£2,686,800","£1,011,290","£251,000","£467,250"
2,Athletics,"£20,613,100","£21,820,808","£22,416,808","£22,175,520"
3,Badminton,"£5,979,860","£4,922,229","£889,793","£3,322,558"
4,Baseball,-,-,"£30,500","£176,250"
5,Basketball,"£7,127,800","£1,927,061","£330,500","£1,350,000"
6,Boxing,"£7,551,900","£11,151,737","£11,606,057","£11,597,507"
7,Breaking,-,-,-,"£135,000"
8,Canoeing,"£12,981,800","£16,293,651","£15,765,693","£13,081,836"
9,Cycling,"£20,613,900","£23,966,916","£24,559,306","£29,172,590"


In [ ]:
# get data from csv

inflation_df = pd.read_csv("InflationData.csv")
inflation_df = inflation_df.loc[30:42] # select relevant years for inflation


# Convert itno float to be useable
inflation_df['CPIH ANNUAL RATE 00: ALL ITEMS 2015=100'] = inflation_df['CPIH ANNUAL RATE 00: ALL ITEMS 2015=100'].astype(float)
inflation_df['Year'] = inflation_df['Title'].astype(int)
# Set intial inflation multiplier to 100
inflation_df.loc[inflation_df['Title'] == '2012', 'Multiplier'] = 100

# Cumulative inflation effect
for i in range(2013, 2025):
    prev_multiplier = inflation_df.loc[inflation_df['Year'] == i-1, 'Multiplier'].values[0]
    curr_inflation = inflation_df.loc[inflation_df['Year'] == i, 'CPIH ANNUAL RATE 00: ALL ITEMS 2015=100'].values[0]
    inflation_df.loc[inflation_df['Year'] == i, 'Multiplier'] = prev_multiplier * (1 + curr_inflation / 100)

inflation_df

,Title,CPIH ANNUAL RATE 00: ALL ITEMS 2015=100,Year,Multiplier
30,2012,2.6,2012,100.000000
31,2013,2.3,2013,102.300000
32,2014,1.5,2014,103.834500
33,2015,0.4,2015,104.249838
34,2016,1.0,2016,105.292336
35,2017,2.6,2017,108.029937
36,2018,2.3,2018,110.514626
37,2019,1.7,2019,112.393374
38,2020,1.0,2020,113.517308
39,2021,2.5,2021,116.355241


In [ ]:
# scrape for GB medal performance
from urllib import request, error
from bs4 import BeautifulSoup

url = 'https://en.wikipedia.org/wiki/Great_Britain_at_the_Olympics'


response = BeautifulSoup(request.urlopen(url), 'html.parser')

tables = response.find_all('table', class_='wikitable')


# Manually find table
TAB = tables[2]
#
rows = TAB.find_all('tr')

medals = []

# Process each row
for row in rows[1:]: 
    cells = row.find_all(['td', 'th'])
    row_contents = [cell.get_text(strip=True).replace(',', '') for cell in cells]
    medals.append(row_contents)


medals_df = pd.DataFrame(medals, columns = ['Games', 'Athletes', 'Gold', 'Silver', 'Bronze', 'Total', 'Rank'])

medals_df = medals_df.loc[26:29]

medals_df

#weighted normalisation?


,Games,Athletes,Gold,Silver,Bronze,Total,Rank
26,2012 London,541,29,18,18,65,3
27,2016 Rio de Janeiro,366,27,23,17,67,2
28,2020 Tokyo,376,22,20,22,64,4
29,2024 Paris,327,14,22,29,65,7
